In [10]:
# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

# We'll use pandas to generate a nice table
import pandas as pd

# We'll graph the functions, so we'll need the matplotlib library.
# We'll also choose the display mode for these graphs. To simplify, we choose
%matplotlib inline
# Now, we'll nickname the pyplot submodule of matplotlib as plt.
import matplotlib.pyplot as plt

# Finally, we'll import some LaTeX and Markdown display helpers
from IPython.display import display, clear_output, Markdown, Latex

# How many minutes?

In TechTown, there are two cell phone service providers. "PhoneWorld" offers a monthly plan, with a \\$35 fee for 120 minutes, plus \\$0.10 each additional minute. "CellUniverse" offers a \\$30 monthly plan for 100 minutes, and each additional minute costs \\$0.15 up to 300 minutes. After that, each extra minute costs \\$0.05. 

How many monthly minutes would you have to use such that "PhoneWorld" is the best option, that is, costs less than "CellUniverse"?

## Setting up the problem (inspection)

First, let's take a look at how we could compute the monthly fee for each cell phone service provider. Usually when looking at problems like this, we try some random values at it. Let's try a table:

In [11]:
def phoneworld(minutes):
    if minutes <= 120:
        cost = 35
    else:
        cost = 35+(minutes-120)*0.1
    return cost

In [12]:
def celluniverse(minutes):
    if minutes <= 100:
        cost = 30
    elif minutes <= 300:
        cost = 30 + (minutes-100)*0.15
    else:
        cost = 30 + 200*0.15 + (minutes-300)*0.05
    return cost

In [13]:
minutes_input = widgets.Text(
                    value='',
                    placeholder='Enter minutes (decimal)',
                    description="Minutes =",
                )

# Initializing table with the first row
data = [['Basic Fee', 35, 30]] 
table = pd.DataFrame(data, columns = ['Minutes', 'PhoneWorld', 'CellUniverse']) 

interact_manual.opts['manual_name'] = 'Compute Cost'

@interact_manual(minutes=minutes_input, table=fixed(table))
def update_table(minutes, table):
    if minutes != '':
        time = float(minutes)
        if len(table[table['Minutes'] == time]) > 0:
            # this value has been computed already, don't do anything
            pass
        else:
            table.loc[len(table)+1] = (time, phoneworld(time), celluniverse(time))
    # we'll clear the table index to have a nicer display
    table.index = ['']*len(table)
    display(table)

interactive(children=(Text(value='', description='Minutes =', placeholder='Enter minutes (decimal)'), Button(d…

Now, it would be interesting to see how these values compare in a graph. In the horizontal axis, we'll mark how many minutes were used, and in the vertical axis we'll mark the corresponding cost for each company.

In [14]:
fig, ax = plt.subplots()
plt.close(fig)

graph_button = widgets.Button(
                description='Graph computed values',
                tooltip='Graph computed values',
                layout=widgets.Layout(width='auto')
                )

connect_button = widgets.Button(
                    description='Connect points',
                    tooltip='Connect points',
                    layout=widgets.Layout(width='auto')
                    )

display(widgets.HBox([graph_button, connect_button]))
out_graphs = widgets.Output()
display(widgets.HBox(children=(out_graphs,)))

def graph_computed(graph):
    ax.clear()
    ax.plot(table['Minutes'][1:], table['PhoneWorld'][1:], 'r*', label="PhoneWorld")
    ax.plot(table['Minutes'][1:], table['CellUniverse'][1:], 'bo', label="CellUniverse")
    ax.legend()
    with out_graphs:
        clear_output(wait=True)
        display(fig)
        
def connect_points(connect):
    ax.clear()
    # Let's reorder the amount of minutes in the table so that our 
    # graph is organized
    in_values = table[1:].sort_values(by='Minutes')
    ax.plot(in_values['Minutes'], in_values['PhoneWorld'], 'r*')
    ax.plot(in_values['Minutes'], in_values['CellUniverse'], 'bo')
    ax.plot(in_values['Minutes'], in_values['PhoneWorld'], 'r')
    ax.plot(in_values['Minutes'], in_values['CellUniverse'], 'b')
    ax.legend(("PhoneWorld", "CellUniverse"))
    with out_graphs:
        clear_output(wait=True)
        display(fig)

graph_button.on_click(graph_computed)
connect_button.on_click(connect_points)

Now, we can see that there is an interval where the red line is lower than the blue line; that means that PhoneWorld is less expensive than CellUniverse only in that interval. Let's see when that is.

From the graph, we can see that the interval that we are interested in is determined by the two points where the blue and the red line intersect. (If you can't see those two points, try adding more values to your table - say, up to 500 minutes). Now, how do we find those points?

## Finding the intersection of the two lines

Both lines are determined by *functions*. These are a way to connect the minutes to their correponding costs for each company. Just to keep it simpler, we'll give a nickname to the monthly cost of each company. The cost for PhoneWorld will be called $W$ and the cost for CellUniverse will be called $U$. 

We can see that both costs are determined by how many minutes we used in each month. So we can say something like this:

In [15]:
table.set_index('Minutes')
for index, row in table[1:].iterrows():
    print('For {0} minutes, we have W({0}) = {1}, U({0}) = {2}.'.format(int(row[0]), row[1], row[2]))

Notice that for values that are not on the table, we can't directly decide how much it would cost, unless we compute this number too. Fortunately, there is an easier way to describe to total cost without us having to list all possible values: using the *law* of the function. In our case, we have the following: let us call $t$ the total amount of minutes used in that month. Then,

In [16]:
# Title
display(Markdown('**For PhoneWorld:**<br/> *(Press Enter after typing)*'))

# Display function name - W(t)
name_phoneworld = widgets.Output(layout=widgets.Layout(min_width='80px'))
with name_phoneworld:
    display(Latex('$W(t)= $'))
    
# Display first interval for the function definition
condition1_phoneworld = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condition1_phoneworld'))
with condition1_phoneworld:
    display(Latex('for $t \leq 120$'))
    
# Display second interval for the function definition
condition2_phoneworld = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condition2_phoneworld'))
with condition2_phoneworld:
    display(Latex('for $t > 120$.'))
    
# Display resulting function graph
fig_phoneworld, ax_phoneworld = plt.subplots()
outgraph_phoneworld = widgets.Output()
plt.close(fig_phoneworld)

def start_fig_phoneworld(fig_phoneworld, ax_phoneworld, outgraph_phoneworld):
    with outgraph_phoneworld:
        values = [0, 120, 500]
        output = []
        for t in values:
            output.append(phoneworld(t))
        ax_phoneworld.plot(values, output, 'r', label='PhoneWorld')
        ax_phoneworld.legend()
        display(fig_phoneworld)

start_fig_phoneworld(fig_phoneworld, ax_phoneworld, outgraph_phoneworld)

# Widgets for the function law input
textbox1_phoneworld = widgets.Text(layout=widgets.Layout(width='auto', grid_area='textbox1_phoneworld'))
textbox2_phoneworld = widgets.Text(layout=widgets.Layout(width='auto', grid_area='textbox2_phoneworld'))

# Widgets are displayed in a grid
grid_phoneworld = widgets.GridBox(
    children=[textbox1_phoneworld, condition1_phoneworld, 
              textbox2_phoneworld, condition2_phoneworld],
    layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
                "textbox1_phoneworld condition1_phoneworld"
                "textbox2_phoneworld condition2_phoneworld"
                ''')
    )

# This widget contains the three previously defined widgets, side by side.
horizontal_box = widgets.HBox(
    children=[name_phoneworld, grid_phoneworld, outgraph_phoneworld],
    layout=widgets.Layout(width='90%', align_items='center', justify_content='center')
    )

# The functions below show the graph of the conditions given by the users 
# in the input boxes defined above. The graphs are then compared to the 
# true function graph, so that we can check that the suggested law actually
# represents the conditions we want to observe for this function.
def graph_phoneworld_condition1(box):
    values = [0, 120]
    output = []
    for t in values:
        # careful with eval!
        output.append(eval(box.value))
    ax_phoneworld.plot(values, output, 'k-', label="My graph ($0\leq t\leq 120$)")
    ax_phoneworld.legend()
    with outgraph_phoneworld:
        clear_output(wait=True)
        display(fig_phoneworld)
        
def graph_phoneworld_condition2(box):
    values = [120, 500]
    output = []
    for t in values:
        output.append(eval(box.value))
    ax_phoneworld.plot(values, output, 'k-', label="My graph ($t>120$)")
    ax_phoneworld.legend()
    with outgraph_phoneworld:
        clear_output(wait=True)
        display(fig_phoneworld)
    
# Finally, we display all the widgets
display(horizontal_box)
textbox1_phoneworld.on_submit(graph_phoneworld_condition1)
textbox2_phoneworld.on_submit(graph_phoneworld_condition2)

# We create a button to clear the graph in case we make a mistake
clear_button_phoneworld = widgets.Button(
                            description='Clear graph',
                            tooltip='Clear graph',
                            layout=widgets.Layout(width='auto')
                            )

def clear_graph_phoneworld(button):
    with outgraph_phoneworld:
        clear_output(wait=True)
        ax_phoneworld.clear()
        start_fig_phoneworld(fig_phoneworld, ax_phoneworld, outgraph_phoneworld)

display(clear_button_phoneworld)
clear_button_phoneworld.on_click(clear_graph_phoneworld)

**For PhoneWorld:**<br/> *(Press Enter after typing)*

Button(description='Clear graph', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Clear graph')

Let's repeat this for CellUniverse (remember that here we have 3 price ranges):

In [17]:
# Title
display(Markdown('**For CellUniverse:**<br/> *(Press Enter after typing)*'))

# Display function name - U(t)
name_celluniverse = widgets.Output(layout=widgets.Layout(min_width='80px'))
with name_celluniverse:
    display(Latex('$U(t)= $'))
    
# Display first interval for the function definition
condition1_celluniverse = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condition1_celluniverse'))
with condition1_celluniverse:
    display(Latex('for $t \leq 100$'))
    
# Display second interval for the function definition
condition2_celluniverse = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condition2_celluniverse'))
with condition2_celluniverse:
    display(Latex('for $100 < t \leq 300$'))

# Display third interval for the function definition
condition3_celluniverse = widgets.Output(layout=widgets.Layout(width='auto', grid_area='condition3_celluniverse'))
with condition3_celluniverse:
    display(Latex('for $t > 300$.'))

# Display resulting function graph
fig_celluniverse, ax_celluniverse = plt.subplots()
plt.close(fig_celluniverse)
outgraph_celluniverse = widgets.Output()

def start_fig_celluniverse(fig_celluniverse, ax_celluniverse, outgraph_celluniverse):
    with outgraph_celluniverse:
        values = [0, 100, 300, 500]
        output = []
        for t in values:
            output.append(celluniverse(t))
        ax_celluniverse.plot(values, output, 'b', label='CellUniverse')
        ax_celluniverse.legend()
        display(fig_celluniverse)

start_fig_celluniverse(fig_celluniverse, ax_celluniverse, outgraph_celluniverse)
    
# Widgets for the function law input
textbox1_celluniverse = widgets.Text(layout=widgets.Layout(width='150px', grid_area='textbox1_celluniverse'))
textbox2_celluniverse = widgets.Text(layout=widgets.Layout(width='150px', grid_area='textbox2_celluniverse'))
textbox3_celluniverse = widgets.Text(layout=widgets.Layout(width='150px', grid_area='textbox3_celluniverse'))

# Widgets are displayed in a grid
grid_celluniverse = widgets.GridBox(
    children=[textbox1_celluniverse, condition1_celluniverse, 
              textbox2_celluniverse, condition2_celluniverse, 
              textbox3_celluniverse, condition3_celluniverse],
    layout=widgets.Layout(
            width='350px',
            grid_template_rows='auto auto',
            grid_template_columns='150px 200px',
            grid_template_areas='''
                "textbox1_celluniverse condition1_celluniverse"
                "textbox2_celluniverse condition2_celluniverse"
                "textbox3_celluniverse condition3_celluniverse"
                ''')
    )

# This widget contains the three previously defined widgets, side by side.
horizontal_box = widgets.HBox(children=[name_celluniverse, grid_celluniverse, outgraph_celluniverse],
                  layout=widgets.Layout(width='100%', align_items='center', justify_content='center'))

# The functions below show the graph of the conditions given by the users 
# in the input boxes defined above. The graphs are then compared to the 
# true function graph, so that we can check that the suggested law actually
# represents the conditions we want to observe for this function.
def graph_celluniverse_condition1(box):
    values = [0, 100]
    output = []
    for t in values:
        output.append(eval(box.value))
    ax_celluniverse.plot(values, output, 'k-', label="My graph ($0\leq t\leq 100$)")
    ax_celluniverse.legend()
    with outgraph_celluniverse:
        clear_output(wait=True)
        display(fig_celluniverse)
        
def graph_celluniverse_condition2(box):
    values = [100, 300]
    output = []
    for t in values:
        output.append(eval(box.value))
    ax_celluniverse.plot(values, output, 'k-', label="My graph ($100<t\leq 300$)")
    ax_celluniverse.legend()
    with outgraph_celluniverse:
        clear_output(wait=True)
        display(fig_celluniverse)

def graph_celluniverse_condition3(box):
    values = [300, 500]
    output = []
    for t in values:
        output.append(eval(box.value))
    ax_celluniverse.plot(values, output, 'k-', label="My graph ($t > 500$)")
    ax_celluniverse.legend()
    with outgraph_celluniverse:
        clear_output(wait=True)
        display(fig_celluniverse)

# Finally, we display all the widgets
display(horizontal_box)
textbox1_celluniverse.on_submit(graph_celluniverse_condition1)
textbox2_celluniverse.on_submit(graph_celluniverse_condition2)
textbox3_celluniverse.on_submit(graph_celluniverse_condition3)

# We create a button to clear the graph in case we make a mistake
clear_button_celluniverse = widgets.Button(
                description='Clear graph',
                tooltip='Clear graph',
                layout=widgets.Layout(width='auto')
                )

def clear_graph_celluniverse(clear):
    with outgraph_celluniverse:
        clear_output(wait=True)
        ax_celluniverse.clear()
        start_fig_celluniverse(fig_celluniverse, ax_celluniverse, outgraph_celluniverse)

display(clear_button_celluniverse)
clear_button_celluniverse.on_click(clear_graph_celluniverse)

**For CellUniverse:**<br/> *(Press Enter after typing)*

Button(description='Clear graph', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='Clear graph')

Now, you may have seen that the laws for W(t) and U(t) are:

In [18]:
show_w = widgets.Output()
show_v = widgets.Output()
with show_w:
    display(Latex(r'$$W(t) = \begin{cases} 35, & t \leq 120\\ 35+0.1(t-120), & t> 120\end{cases}$$'))
with show_v:
    display(Latex(r'$$U(t) = \begin{cases} 30, & t\leq 100\\30+0.15(t-100), &100<t\leq 300\\ 30 + 200(0.15) + 0.05(t-300), & t>300.\end{cases}$$'))

# We'll use an accordion widget to keep the contents hidden until the user
# decides to click them, avoiding "spoilers" :)
# The option "selected_index=None" makes sure no option comes pre-selected.
accordion = widgets.Accordion(children=[show_w, show_v], selected_index=None)
accordion.set_title(0, 'PhoneWorld:')
accordion.set_title(1, 'CellUniverse:')
display(accordion)

Accordion(children=(Output(), Output()), selected_index=None, _titles={'0': 'PhoneWorld:', '1': 'CellUniverse:…

In Python, we would do something like this: for PhoneWorld, we have

In [19]:
def W(t):
    if t <= 120:
        W = 35
    else:
        W = 35+(t-120)*0.1
    return W

and for CellUniverse, we have

In [20]:
def U(t):
    if t <= 100:
        U = 30
    elif t <= 300:
        U = 30 + (t-100)*0.15
    else:
        U = 30 + 200*0.15 + (t-300)*0.05
    return U

Finally, to find the intersection points between the two curves, we can see that for the leftmost point, we are in the interval where $100<t<300$, and so we will use the second law for $W$ and the second law for $U$:

\begin{align*}
    35+0.1(t-120) &= 30+0.15(t-100)\\
    35 + 0.1t - 12 &= 30+0.15t - 15\\
    0.1t-0.15t &= 30-15-35+12\\
    -0.05t &= -8
\end{align*}
and so,
$$t = \frac{800}{5} = 160.$$

In fact, we can check that this is true: $W(160)$ is 

In [21]:
W(160)

39.0

and

In [22]:
U(160)

39.0

Next, we have a point where $400<t<500$, and so we must have

\begin{align*}
    35+0.1(t-120) &= 30 + 200(0.15) + 0.05(t-300)\\
    35 + 0.1t - 12 &= 30 + 30 + 0.05t - 15\\
    0.1t-0.05t &= 30+30-15-35+12\\
    0.05t &= 22
\end{align*}
and so,
$$t = \frac{2200}{5} = 440.$$

Indeed,

In [23]:
W(440)

67.0

and

In [24]:
U(440)

67.0

## Answer

For PhoneWorld to be the best option, our monthly usage must be between 160 and 440 minutes.